In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.2.4'

In [2]:
#Using a pre-trained model
#There are two ways to leverage a pre-trained network: 
#feature extraction and fine-tuning. We will cover both of them. Let's start with feature extraction.

In [3]:
from keras.applications import VGG16
conv_base = VGG16(weights = 'imagenet' , include_top = False , input_shape = (150 , 150 ,3)) 

58892288/58889256 [==============================] - 34s 1us/step


In [4]:
#include_top specifies whether to include Dense layers or not
#weights specifies the type of data in which VGG16 has trained

In [6]:
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [7]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

base_dir = 'C:/dev/python_projects/data/cats_and_dogs_small'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

In [9]:
def extract_features(directory , sample_count):
    features = np.zeros(shape = (sample_count , 4 , 4 , 512))
    labels = np.zeros(shape = (sample_count))
    generator = datagen.flow_from_directory(directory , target_size = (150 ,150) , batch_size = batch_size , class_mode = 'binary')
    i = 0
    for input_batch , labels_batch in generator:
        features_batch = conv_base.predict(input_batch)
        features[i * batch_size : (i+1)*batch_size] = features_batch
        labels[i * batch_size : (i+1)*batch_size] = labels_batch
        i += 1
        if i*batch_size >= sample_count:
            break
    return features , labels
        
train_features, train_labels = extract_features(train_dir, 2000)
validation_features, validation_labels = extract_features(validation_dir, 1000)
test_features, test_labels = extract_features(test_dir, 1000)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [13]:
train_features = np.reshape(train_features, (2000, 4 * 4 * 512))
validation_features = np.reshape(validation_features, (1000, 4 * 4 * 512))
test_features = np.reshape(test_features, (1000, 4 * 4 * 512))


In [15]:
from keras import models
from keras import layers
from keras import optimizers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=4 * 4 * 512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
              loss='binary_crossentropy',
              metrics=['acc'])

history = model.fit(train_features, train_labels,
                    epochs=30,
                    batch_size=20,
                    validation_data=(validation_features, validation_labels))

Train on 2000 samples, validate on 1000 samples
Epoch 1/30
2000/2000 [==============================] - 11s 6ms/step - loss: 0.5793 - acc: 0.6865 - val_loss: 0.4427 - val_acc: 0.8240
Epoch 2/30
2000/2000 [==============================] - 10s 5ms/step - loss: 0.4221 - acc: 0.8065 - val_loss: 0.3559 - val_acc: 0.8680
Epoch 3/30
2000/2000 [==============================] - 10s 5ms/step - loss: 0.3639 - acc: 0.8415 - val_loss: 0.3266 - val_acc: 0.8690
Epoch 4/30
2000/2000 [==============================] - 10s 5ms/step - loss: 0.3050 - acc: 0.8840 - val_loss: 0.2957 - val_acc: 0.8860
Epoch 5/30
2000/2000 [==============================] - 9s 5ms/step - loss: 0.2905 - acc: 0.8795 - val_loss: 0.2857 - val_acc: 0.8970
Epoch 6/30
2000/2000 [==============================] - 9s 5ms/step - loss: 0.2644 - acc: 0.8960 - val_loss: 0.2748 - val_acc: 0.9020
Epoch 7/30
2000/2000 [==============================] - 5s 2ms/step - loss: 0.2465 - acc: 0.9015 - val_loss: 0.2623 - val_acc: 0.9010
Epoch 8/30

In [16]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [ ]:
#more at
#http://nbviewer.jupyter.org/github/fchollet/deep-learning-with-python-notebooks/blob/master/5.3-using-a-pretrained-convnet.ipynb